In [1]:
import numpy as np
import os
import pandas as pd
import re

In [2]:
episodes = []
for (dirpath, dirnames, filenames) in os.walk('C:\\Users\\Nirvan S P Theethira\\Desktop\\Mimic\\data\\transcripts_friends\\season_all\\'):
    for file in filenames:
        if '.csv' in file:
            episodes.append(dirpath+file)

In [3]:
def readFile(lines):
    scene = ''
    preChar = ''
    preScene = ''
    dialogue = ''
    # dataF = {'scene':[],'person':[], 'dialogue':[]}
    dataF = []
    dataRaw = ''
    prevDialogue = ''
    for line in lines:
        line = re.sub('\n','',line).lower()
        if re.search(r"^\[[\w\W]+\]$", re.sub('\n','',line)):
            scene = line
        if re.search('^[a-z]+:',line) and len(scene)>0:
            charachter = re.findall('^[a-z]+:',line)[0]
            charachter = re.sub(':','',charachter).upper()
            if preChar!=charachter and len(preChar)!=0:
                if preChar=='JOEY':
                    dialogue = re.sub('\((.*?)\)','',dialogue)
                    dialogue = re.sub('^\s+','',dialogue)
                    dialogue = re.sub('\s+$','',dialogue)
                    dataRaw += prevDialogue+'\t'+dialogue+'\n'
                dataF.append([preChar.upper(),dialogue])
                prevDialogue = re.sub('\((.*?)\)','',dialogue)
                prevDialogue = re.sub('^\s+','',prevDialogue)
                prevDialogue = re.sub('\s+$','',prevDialogue)
                dialogue=''

            dialogue += re.sub('^[a-z]+:','',line)
            preChar = charachter
    return dataRaw

In [4]:
allJoey = ''
for episode in episodes[0:-2]:
    print(episode)
    with open(episode,encoding='utf8') as f:
        lines = f.readlines()
        raw = readFile(lines)
        allJoey += raw

C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1001.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1002.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1003.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1004.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1005.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1006.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1007.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1008.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1009.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\101.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\1010.csv
C:\Users\Nirvan S P Th

C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\503.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\504.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\505.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\506.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\507.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\508.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\509.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\510.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\511.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\512.csv
C:\Users\Nirvan S P Theethira\Desktop\Mimic\data\transcripts_friends\season_all\513.csv
C:\Users\Nirvan S P Theethira\De

In [5]:
np.save('joey',allJoey)